# Coursera IBM Data Analysis Capstone
#### Paolo Coraggio

## Week 3 Assignment

> In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

#### Part 3 Assignment

**Explore and cluster** the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

- to add enough Markdown cells to explain what you decided to do and to report any observations you make.
- to generate maps to visualize your neighborhoods and how they cluster together.

Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)

#### Part 3 Submission

Importing relevant libraries and data

In [9]:
import pandas as pd
import numpy as np

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

dfdata = pd.read_csv('./data/dfdatalatlong.csv', index_col = 0)

In [10]:
dfdata.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Retrieving Toronto's geographical coordinates

As first step, I retrieve Toronto's geographical coordinates to display its map

In [11]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Then I diplay the map of Toronto with the labelled Neighborhood

In [12]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10,
                        min_zoom=9,max_zoom=15)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfdata['Latitude'], 
                                           dfdata['Longitude'], 
                                           dfdata['Borough'], 
                                           dfdata['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Mapping the locations in Central Toronto

Now I create a data frame containing only the neighborhood in Central Toronto by selecting the Borough containing the string `Toronto`

In [13]:
df_Toronto_Toronto = dfdata[dfdata['Borough'].str.contains('Toronto')].reset_index(drop = True)
df_Toronto_Toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Now I visualize these locations on the map

In [14]:
map_toronto_t = folium.Map(location=[latitude, longitude], zoom_start=11,
                        min_zoom=10,max_zoom=15)

for lat, lng, borough, neighborhood in zip(df_Toronto_Toronto['Latitude'], 
                                           df_Toronto_Toronto['Longitude'], 
                                           df_Toronto_Toronto['Borough'], 
                                           df_Toronto_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#F74A69',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_t)  
    
map_toronto_t

#### Define Foursquare Credential and Version

In [15]:
CLIENT_ID = 'TLMY2YZJF321RW0TNUFKSSIZK22CWMWSUVLDTSWRSDI25BYH' # your Foursquare ID
CLIENT_SECRET = 'OCDTNIQ5GSSCIXFVC303JZFQNC4T2FQSZG1GIYZG12FTTWEQ' # your Foursquare Secret
VERSION = '20200623' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TLMY2YZJF321RW0TNUFKSSIZK22CWMWSUVLDTSWRSDI25BYH
CLIENT_SECRET:OCDTNIQ5GSSCIXFVC303JZFQNC4T2FQSZG1GIYZG12FTTWEQ


## Explore Neighborhoods in Toronto Central

I first define two functions for

- Define the category type (`get_category_type`)
- Iteratively get the nerby venues of a given location (`getNearbyVenues`)

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now we retrieve the top 100 venues for each location in central Toronto.

> **Note**: in the course exercise the first location of Toronto dataframe is examined. In this exercise I skip this part as I am interested in the clustering of all locations.

In [18]:
radius = 500
LIMIT = 100

ttoronto_venues = getNearbyVenues(names= df_Toronto_Toronto['Neighborhood'],
                              latitudes= df_Toronto_Toronto['Latitude'],
                              longitudes= df_Toronto_Toronto['Longitude']
                             )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [19]:
ttoronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub


Let's check how many venues were returned for each neighborhood and display the first 5 entries

In [20]:
ttoronto_counts = ttoronto_venues.groupby('Neighborhood').count()
ttoronto_counts.head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",20,20,20,20,20,20
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,65,65,65,65,65,65


And now I list how many unique categories can be curated from all the returned venues.

In [27]:
print('There are {} uniques categories.'.format(len(ttoronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


## Analyse each neighborhood

In [28]:
# one hot encoding
ttoronto_onehot = pd.get_dummies(ttoronto_venues[['Venue Category']], prefix="", prefix_sep="")
ttoronto_onehot.rename(columns={'Neighborhood':'Neighborhood (category)'}, inplace=True)

# add neighborhood column back to dataframe
ttoronto_onehot['Neighborhood'] = ttoronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ttoronto_onehot.columns[-1]] + list(ttoronto_onehot.columns[:-1])
ttoronto_onehot = ttoronto_onehot[fixed_columns]

ttoronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
ttoronto_onehot.shape

(1616, 234)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [29]:
ttoronto_grouped = ttoronto_onehot.groupby('Neighborhood').mean().reset_index()
print(ttoronto_grouped.shape)
ttoronto_grouped.head()

(39, 234)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.050000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.071429,0.071429,0.071429,0.142857,0.142857,0.071429,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
4,Central Bay Street,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.015385,0.0,0.0,0.015385,0.0,0.0,0.015385


To check that we get results that makes sens, I print each neighborhood along with the top 3 most common venues

In [30]:
num_top_venues_ttoronto = 3

for hood in ttoronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ttoronto_grouped[ttoronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues_ttoronto))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2      Beer Bar  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.10
1         Yoga Studio  0.05
2                Park  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0   Airport Lounge  0.14
1  Airport Service  0.14
2  Harbor / Marina  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1      Sandwich Place  0.06
2  Italian Restaurant  0.06


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12


----Church and Wellesley----
                 venue  freq
0    

Now I insert the data into a pandas dataframe. First, let's write a function to sort the venues in descending order.

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

And create the new dataframe and display the top 10 venues for each neighborhood.

In [32]:
num_top_venues_ttoronto = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues_ttoronto):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_tt = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_tt['Neighborhood'] = ttoronto_grouped['Neighborhood']

for ind in np.arange(ttoronto_grouped.shape[0]):
    neighborhoods_venues_sorted_tt.iloc[ind, 1:] = return_most_common_venues(ttoronto_grouped.iloc[ind, :], 
                                                                             num_top_venues_ttoronto)

neighborhoods_venues_sorted_tt.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Restaurant,Café,Cheese Shop,Beer Bar,Department Store,Diner
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Performing Arts Venue,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Bakery
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Auto Workshop,Comic Shop,Park,Pizza Place,Moving Target,Recording Studio,Restaurant,Burrito Place,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Harbor / Marina,Boat or Ferry,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Airport Gate,Airport Terminal
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Japanese Restaurant,Café,Thai Restaurant,Salad Place,Bar,Burger Joint,Bubble Tea Shop


## Cluster Neighborhood

Run k-means to cluster the neighborhood into 5 clusters

In [33]:
# set number of clusters
kclusters = 5

ttoronto_grouped_clustering = ttoronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans_tt = KMeans(n_clusters=kclusters, random_state=0).fit(ttoronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans_tt.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [34]:
kmeans_tt.labels_

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 1, 2, 0, 2,
       2, 2, 2, 2, 0, 4, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2])

And create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [35]:
# add clustering labels
neighborhoods_venues_sorted_tt.insert(0, 'Cluster Labels', kmeans_tt.labels_)
#neighborhoods_venues_sorted
ttoronto_merged = df_Toronto_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ttoronto_merged_ = ttoronto_merged.join(neighborhoods_venues_sorted_tt.set_index('Neighborhood'), on='Neighborhood')

In [36]:
ttoronto_merged_.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Theater,Cosmetics Shop,Shoe Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Diner,Sushi Restaurant,Yoga Studio,Park,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Café,Bubble Tea Shop,Diner,Fast Food Restaurant,Italian Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,American Restaurant,Restaurant,Gastropub,Cocktail Bar,Japanese Restaurant,Hotel,Gym,Clothing Store
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Neighborhood (category),Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio


In [37]:
# create map
map_clusters_tt = folium.Map(location=[latitude, longitude], zoom_start=11,
                        min_zoom=10,max_zoom=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ttoronto_merged_['Latitude'], ttoronto_merged_['Longitude'],
                                  ttoronto_merged_.index, ttoronto_merged_['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_tt)
       
map_clusters_tt

The map above shows that in central Toronto the neighborhoods are quite homogeneus in the sense that in the central part of the city all venues looks similar with some exceptions.

## Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Cluster 1

In [38]:
ttoronto_merged_.loc[ttoronto_merged_['Cluster Labels'] == 0, ttoronto_merged_.columns[[1] + list(range(5, ttoronto_merged_.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,0,Trail,Health Food Store,Neighborhood (category),Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
29,Central Toronto,0,Park,Tennis Court,Trail,Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Deli / Bodega
33,Downtown Toronto,0,Park,Playground,Trail,Deli / Bodega,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


#### Cluster 2

In [39]:
ttoronto_merged_.loc[ttoronto_merged_['Cluster Labels'] == 1, ttoronto_merged_.columns[[1] + list(range(5, ttoronto_merged_.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,1,Park,Swim School,Bus Line,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop


#### Cluster 3

In [40]:
ttoronto_merged_.loc[ttoronto_merged_['Cluster Labels'] == 2, ttoronto_merged_.columns[[1] + list(range(5, ttoronto_merged_.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Theater,Cosmetics Shop,Shoe Store
1,Downtown Toronto,2,Coffee Shop,Diner,Sushi Restaurant,Yoga Studio,Park,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place
2,Downtown Toronto,2,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Café,Bubble Tea Shop,Diner,Fast Food Restaurant,Italian Restaurant
3,Downtown Toronto,2,Coffee Shop,Café,American Restaurant,Restaurant,Gastropub,Cocktail Bar,Japanese Restaurant,Hotel,Gym,Clothing Store
5,Downtown Toronto,2,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Restaurant,Café,Cheese Shop,Beer Bar,Department Store,Diner
6,Downtown Toronto,2,Coffee Shop,Italian Restaurant,Sandwich Place,Japanese Restaurant,Café,Thai Restaurant,Salad Place,Bar,Burger Joint,Bubble Tea Shop
7,Downtown Toronto,2,Grocery Store,Café,Park,Italian Restaurant,Coffee Shop,Nightclub,Diner,Candy Store,Baby Store,Restaurant
8,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Gym,Deli / Bodega,Thai Restaurant,Hotel,Pizza Place,Clothing Store,Bookstore
9,West Toronto,2,Bakery,Pharmacy,Brewery,Pizza Place,Bar,Middle Eastern Restaurant,Café,Supermarket,Music Venue,Pet Store
10,Downtown Toronto,2,Coffee Shop,Aquarium,Café,Hotel,Italian Restaurant,Restaurant,Brewery,Sporting Goods Shop,Scenic Lookout,Fried Chicken Joint


#### Cluster 4

In [41]:
ttoronto_merged_.loc[ttoronto_merged_['Cluster Labels'] == 3, ttoronto_merged_.columns[[1] + list(range(5, ttoronto_merged_.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,3,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


#### Cluster 5

In [42]:
ttoronto_merged_.loc[ttoronto_merged_['Cluster Labels'] == 4, ttoronto_merged_.columns[[1] + list(range(5, ttoronto_merged_.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,Garden,Home Service,Pool,Deli / Bodega,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


## Final remarks

From the clustering results, we can see that Toronto Central has mainly a young/wealthy neighborhood as the main venues are Coffes, restaurants and places where to eat or hang out (**cluster 3**).

The other clusters are smaller and looks characterised by:

- Upper/wealthy neighborhood (**cluster 1**): Expensive restaurants and sport leisure
- Family/wealthy neighborhood (**cluster 2**): Parks, fast food restaurants, schools
- Upper/very wealthy neighborhood (**cluster 4**): Jewelry store and ethnic restaurants/leisure
- Middle class neighborhood (**cluster 5**): Garden, deli and electronics stores

This is a very superficial look as a larger database containing more Borough could get more insights.